In [2]:
# import warnings
# warnings.filterwarnings('ignore')

import ipywidgets as ipw
import csv
from IPython.core.display import HTML, display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
from time import sleep
from functions import * # supplementary file with needed functions

# !jupyter nbextension enable --py widgetsnbextension 
# --sys-prefix
# !jupyter serverextension enable voila
# --sys-prefix

In [3]:
%%html
<link rel="shortcut icon" class="js-site-favicon" type="image/png" href="/pictures/logo.png" />

In [8]:
%%html
<style>
.font_size19 > .widget-label {
    font-style: bold;
    color: black;
    font-size: 19px;
}
.font_size20 > .widget-label {
    font-style: bold;
    color: black;
    font-size: 20px;
}
.widget-dropdown > select {
    font-size: 18px;
    font-style: italic;
}
.widget-readout {
    font-size: 19px;
    font-style: bold;
}
.jupyter-button {
    font-size: 20px;
    font-style: bold;
}
.jp-RenderedHTMLCommon table {
    font-size: 18px;
}

a:link {
  color: blue;
  background-color: transparent;
  text-decoration: none;
}

a:visited {
  color: purple;
  background-color: transparent;
  text-decoration: none;
}

a:hover {
  color: red;
  background-color: transparent;
  text-decoration: underline;
}

a:active {
  color: yellow;
  background-color: transparent;
  text-decoration: underline;
}
</style>

In [42]:
# Open picture with the epidemic diagram of the model
diagramFile = open("pictures/epidemicDiagram.png", "rb")
imgSIYRD = diagramFile.read()
imgSIYRD = ipw.Image(value=imgSIYRD,format='png',width = 400, height = 450)
# Create introduction paragraph withe model explanation and stuff
pghIntro = ipw.HTML("<p align=justify style=font-size:1.3em>Welcome to the S<sup>2</sup>IYRD model simulator. S<sup>2</sup>IYRD is a \
compartmental model of infection propagation with vaccination and reinfection, which allows the user to analyse \
the effect that variations on the rates of these two processes have on the incidence of the disease and on the \
number of fatalities. Optimal protocols of vaccine administration depend on a number of variables such as the \
network of contacts between infected and susceptible individuals or the demographic structure of the affected \
population, which vary widely between countries. Find all details about S<sup>2</sup>IYRD and the model behind \
this simulator in our article published in \
<a href=https://iopscience.iop.org/article/10.1088/1751-8121/accda3 target=_blank>Journal of Physics A.</a> \
The data are available in the github repository of the \
<a href=https://github.com/IkerAtienza/SIYRD target=_blank> SIYRD project.</a><br><br> \
<em>As any compartmental model, S<sup>2</sup>IYRD is a simplified representation of reality. Its usefulness \
relies on its ability to compare the effect of variations in model parameters within the modelling framework. The \
simulations it yields, given a set of effective parameter values, are qualitative and should not be used to make \
predictions or take medical decisions.</em><br><br> \
In our model the population is assigned to five compartments with their corresponding labels: S-Susceptible, \
I-Infected, Y-Reinfected, R-Recovered and D-Dead. People may progress between compartments guided by a set \
of ordinary differential equations (see Ref. to manuscript for more details). The picture on the right \
represents the epidemic diagram of the model. The green dashed lines link the classes whose interaction \
triggers the transitions shown in grey. In our model, susceptible individuals leave their compartment \
through infection or vaccination. Infected individuals can recover or die, regardless of whether they \
suffer a primary infection or a reinfection. Recovered individuals come from two different origins: (i) \
Those individuals who overcome the disease after an infection (ii) Susceptible individuals who are \
vaccinated. On the other hand, recovered individuals can reinfect by interacting with both kinds of \
infected individuals, to recover again or to die. The population is split into two groups, setting an age \
limit, to highlight the effect on disease that different relationships between vaccine administration and \
various demographic structures have. Interactions occur between individuals from both groups interchangeably, \
so that a susceptible individual from group 2 is able to interact with infected individuals from both groups. \
However, transitions are exclusive within the group. Vaccination is implemented in this model by means of a \
parameter v representing the fraction of population vaccinated per unit of time, which in our case, has been \
set to one day. With respect to the vaccination protocol, we have implemented two different scenarios: (i) \
Priority vaccination in which one of the two population groups is vaccinated first at a constant rate \
proportional to the total population, until the fraction of susceptible individuals reaches a certain \
threshold (e.g. 70%). When this occurs, vaccination of the non-prioritized group is initiated as long as \
the previous threshold has not been reached in that group. (ii) Simultaneous vaccination, the two groups \
are vaccinated at the same time at a rate proportional to the size of their respective groups. If vaccination \
ends in one of the two groups first, due to the immunity threshold, the other group picks up all doses, if the \
immunity threshold has not been reached.<br><br>\
With this tool, the user is free to choose the geographic region, i.e. the demographic structure of the \
population for which the simulation is performed, thus establishing a specific contacts pattern; the age \
threshold by which the original population is divided; the value of the parameters driving the transitions \
between compartments, so that it is possible to evaluate different diseases; the vaccination scenario, and \
the daily vaccination rate. Finally, the results of the simulations are evaluated in terms of total reduction \
of fatalities and infections.</p>")

# Create introduction box with both widgets
introBox = ipw.HBox([pghIntro, imgSIYRD],layout={'align_items': 'center'})

In [ ]:
# Demographics section header
hdrDemographics = ipw.HTML("<h2 style=color:#004c98;font-size:2.5em>Demographics</h2>")

# Open file with countries implemented
countries = []
with open('infiles/countries.txt','r') as countriesFile:
    rows = csv.reader(countriesFile, delimiter=',')
    for row in rows:
        countries.append(str(row[0])) 

countries.sort()

# Country choice widget with its associated function
choiceCountry = ipw.Dropdown(
    options= countries,
    description='Country:',
    placeholder='Select',
    value = 'Spain',
)

plotPyr = interactive(plot_population_pyramid, country = choiceCountry)

# Age threshold widgets, linked, with their associated functions
age1 = ipw.IntSlider(min=10, max=80, step=10, value=80, description='Age limit: ' ,continuous_update=False,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray')) 
age2 = ipw.IntSlider(min=10, max=80, step=10, value=80, description='Age limit: ' ,continuous_update=False,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'))
link = ipw.link((age1, 'value'),(age2, 'value'))

plotIFR = interactive(plot_ifr_contacts,{'manual': False, 'manual_name': 'Show IFR & Contacts'},x=age1,country=choiceCountry)
plotDensity = interactive(plot_groups_density,{'manual': False, 'manual_name': 'Show Demographics'}, x=age2, country=choiceCountry)

# Change font-size
w0 = age1.add_class("font_size19")
w0 = age2.add_class("font_size19")
w1 = choiceCountry.add_class("font_size19")

In [ ]:
# User quick guide
hdrGuide = ipw.HTML("<h2 style=color:#004c98;font-size:2.5em>Quick Start</h2>")

hdrStep1 = ipw.HTML("<h2>Step 1: Select country</h2>")
pghStep1 = ipw.HTML("<p align=justify style=font-size:1.3em>By selecting a country, the demographic profile is \
updated. Thus, the contact matrix, the infection fatality risk (IFR) information and specific population abundance are established. However, \
no parameter is automatically updated in the parameters section even if the graphical representations change. All \
countries have their own data, with the exception of India, whose IFR data are those of the Spanish population.\
</p>")
intro1 = ipw.VBox([hdrStep1,pghStep1], layout={'width': '24.5%','align_items': 'center'})

hdrStep2 = ipw.HTML("<h2>Step 2: Select age threshold</h2>")
pghStep2 = ipw.HTML("<p align=justify style=font-size:1.3em>The age threshold divides the original population \
into two groups: G<sub>1</sub> includes all individuals below the established age limit, while G<sub>2</sub> \
includes all those above it. Each group has its own IFR, which determines the recovery and fatality rates, and \
its own abundance, which determines the fraction of individuals in the simulation with those parameters.</p>")
intro2 = ipw.VBox([hdrStep2,pghStep2],layout={'width': '24.5%','align_items': 'center'})

hdrStep3 = ipw.HTML("<h2>Step 3: Select parameters</h2>")
pghStep3 = ipw.HTML("<p align=justify style=font-size:1.3em>The user can freely interact with the parameters \
although some indications are necessary. Infection rates are constant for both groups. Default parameters are \
fixed for Spain (country) and 80 (age threshold). If the user selects any other combination, the recovery and \
fatality rates must be manually updated according to the values displayed in the demographics section. Thus, the \
parameters of the model correspond to those of the chosen country.</p>")
intro3 = ipw.VBox([hdrStep3,pghStep3],layout={'width': '24.5%','align_items': 'center'})

hdrStep4 = ipw.HTML("<h2>Step 4: Run simulation</h2>")
pghStep4 = ipw.HTML("<p align=justify style=font-size:1.3em>Before pressing the run button, the user must select \
a vaccination strategy. To simulate any scenario, other than the <i>No vaccination</i> strategy, the vaccination \
rate must be non-zero. Otherwise, the <i>No Vaccination</i> scenario will be simulated. To perform a different \
simulation, change any of the parameters (e.g. country, vaccination rate/strategy, age limit) and press the run \
button again.</p>")
intro4 = ipw.VBox([hdrStep4,pghStep4],layout={'width': '24.5%','align_items': 'center'})

# Quick start box
sectGuide = ipw.HBox([intro1,intro2,intro3,intro4])
sectGuide.layout = ipw.Layout(width='98%',grid_gap='80px',align_items='flex-start',border="solid 3px #7f003f", padding="35px") 

In [ ]:
# Parameters section header
hdrParams= ipw.HTML("<h2 style=color:#004c98;font-size:2.5em>Parameters</h2>")

# Headers are defined for each group of parameters
hdrInfRates = ipw.HTML("<h2>Infection rates</h2>")
hdrRecRates = ipw.HTML("<h2>Recovery rates</h2>")
hdrDeathRates = ipw.HTML("<h2>Fatality rates</h2>")
hdrVacRates = ipw.HTML("<h2>Vaccination</h2>")

# Infection rates widgets
bsi = ipw.FloatSlider(description=r'\( \beta_{SI}\)',value=0.077,min=0,max=0.25,step=0.005,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.3f')
bsy = ipw.FloatSlider(description=r'\( \beta_{SY}\)',value=(0.5*0.077),min=0,max=0.25,step=0.005,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.3f')
bri = ipw.FloatSlider(description=r'\( \beta_{RI}\)',value=(0.01*0.077),min=0,max=0.25,step=(0.01*0.005),style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')
bry = ipw.FloatSlider(description=r'\( \beta_{RY}\)',value=(0.5*0.01*0.077),min=0,max=0.25,step=(0.01*0.005),style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')

# Recovery rates widgets
r1=ipw.FloatSlider(description=r'\( r_{1}\)',value=0.07626,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')
r2=ipw.FloatSlider(description=r'\( r_{2}\)',value=0.06211,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')

# Fatality rates widgets
mui1 = ipw.FloatSlider(description=r'\( \mu_{I_1}\)',value=0.00067,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')
mui2 = ipw.FloatSlider(description=r'\( \mu_{I_2}\)',value=0.01481,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')
muy1 = ipw.FloatSlider(description=r'\( \mu_{Y_1}\)',value=0,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')
muy2 = ipw.FloatSlider(description=r'\( \mu_{Y_2}\)',value=0,min=0,max=0.07701,step=0.00001,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.5f')

# Vaccination process widgets
vaccStrategy = ipw.Dropdown(
    options=[('No vaccination',0),(u'G\u2081'+' priority', 1), (u'G\u2082'+' priority', 2), ('Simultaneous', 3), ('All', 4)],
    value=0,
    description='Scenario:',
    placeholder='Select',
    style=ipw.SliderStyle(description_width='auto'),
)

vaccRate = ipw.FloatSlider(description="Rate: ",value=0,min=0,max=1,step=0.05,style=ipw.SliderStyle(description_width='auto',handle_color='lightgray'),redout=True,readout_format='.2f')


# Infection rates layout (2x2)
infRatesBox = ipw.TwoByTwoLayout(
        top_left= bsi,
        top_right= bri,
        bottom_left= bsy,
        bottom_right= bry
)
infRatesBox = ipw.VBox([hdrInfRates,infRatesBox], layout={'align_items': 'center', 'padding':'20px'})

# Recovery rates layout (1x2)
recRatesBox = ipw.HBox([r1,r2])
recRatesBox = ipw.VBox([hdrRecRates,recRatesBox], layout={'align_items': 'center', 'padding':'20px'})

# Fatality rates layout (2x2)
deathRatesBox = ipw.TwoByTwoLayout(
        top_left= mui1,
        top_right= mui2,
        bottom_left= muy1,
        bottom_right= muy2
)
deathRatesBox = ipw.VBox([hdrDeathRates,deathRatesBox], layout={'align_items': 'center', 'padding':'20px'})

# Vaccination options layout (1x2)
vaccBox = ipw.HBox([vaccRate, vaccStrategy], layout={'align_items': 'initial'})
vaccBox = ipw.VBox([hdrVacRates, vaccBox], layout={'align_items': 'center', 'padding':'20px'})

# Change font-size
w2 = bsi.add_class("font_size20")
w2 = bsy.add_class("font_size20")
w2 = bri.add_class("font_size20")
w2 = bry.add_class("font_size20")
w3 = r1.add_class("font_size20")
w3 = r2.add_class("font_size20")
w4 = mui1.add_class("font_size20")
w4 = mui2.add_class("font_size20")
w4 = muy1.add_class("font_size20")
w4 = muy2.add_class("font_size20")
w5 = vaccStrategy.add_class("font_size19")
w5 = vaccRate.add_class("font_size19")

# Create parameters box with all the parameters
paramsBox = ipw.VBox([infRatesBox,recRatesBox,deathRatesBox,vaccBox], layout={'align_items': 'center'})

In [ ]:
transitionsFile = open("pictures/transitionsSIYRD.png", "rb")
imgTransitions = transitionsFile .read()
imgTransitions = ipw.Image(value=imgTransitions,format='png',width = 600, height = 450)

paramsBox = ipw.HBox([imgTransitions,paramsBox],layout={'align_items': 'center'})
paramsBox.layout = ipw.Layout(grid_gap='150px',justify_content='space-around')

In [ ]:
# Run simulation button
btnRun = ipw.Button(description='Run simulation',style=ipw.ButtonStyle(description_width='auto',font_weight='bold',button_color='#00cb66'))
outputRun = ipw.Output()

# Function that simulates SIYRD given all the parameters values located in the paramsBox
def runSimulation(b):
    with outputRun:
        clear_output() # delete previous output
        deleteOutfiles() # delete all previous simulation results
        outFiles = [] # save outFiles names
        
        if vaccStrategy.value != 4:
            # if the user forget to change the vacc rate, automatically we set it back to 0            
            if vaccStrategy.value == 0:
                vaccRate.value = 0
            if vaccRate.value == 0:
                vaccStrategy.value = 0
    
            if vaccStrategy.value != 0:
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,vaccStrategy.value,vaccRate.value==0,age1.value)
                outFiles.append(outFile)
                
            outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,vaccStrategy.value,vaccRate.value,age1.value)
            outFiles.append(outFile)
        else:
            # if the user forget to change the vacc rate, automatically we set it back to 0
            if vaccRate.value == 0: 
                vaccStrategy.value = 0
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,vaccStrategy.value,vaccRate.value,age1.value)
                outFiles.append(outFile)
            else:
                # ALL scenario was selected. The four strategies are simulated under the same combination of parameters
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,0,0,age1.value)
                outFiles.append(outFile)
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,1,vaccRate.value,age1.value)
                outFiles.append(outFile)
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,2,vaccRate.value,age1.value)
                outFiles.append(outFile)
                outFile = agesiyrd(choiceCountry.value,bsi.value,bri.value,bsy.value,bry.value,r1.value,r2.value,mui1.value,mui2.value,muy1.value,muy2.value,3,vaccRate.value,age1.value)
                outFiles.append(outFile)
        
        # Print out simulation results as pandas dataframe
        summaryFile = resultsAnalysis(age1.value, vaccRate.value, outFiles)
        dfSummary = pd.read_csv('outfiles/'+summaryFile, sep='\t', thousands=',', index_col='Vaccination strategy')
        dfSummary = dfSummary.round(decimals=2)
        
        if vaccStrategy.value == 4:
            dfSummary = dfSummary.style.highlight_max(subset=['Infection reduction (%)','Mortality reduction (%)', 'Vaccination coverage (%)'],color='lightgreen').set_properties(**{'text-align': 'center'}).format('{:,}')
        else:
            dfSummary = dfSummary.style.set_properties(**{'text-align': 'center'}).format('{:,}')
        

        display(dfSummary)

# Fucntion runSimulation is executed when the button is clicked
btnRun.on_click(runSimulation)

In [ ]:
# Web structure is created

# First element: Web title centered
webTitle = ipw.HTML("<h1 style=color:#004c98;font-size:4em><b> S<sup>2</sup>IYRD MODEL SIMULATOR</b></h1>")

webTitle = ipw.VBox([webTitle], layout={'align_items':'center'})

# Second element: Line
line = ipw.HTML("<hr style=height:5px;width:95%;border-width:0;background-color:#7f003f>")

# Third element: Demographics Box
plot1 = ipw.VBox([plotIFR])
plot2 = ipw.VBox([plotPyr])
plot3 = ipw.VBox([plotDensity])
sectDemography = ipw.HBox([plot2,plot3,plot1])
sectDemography.layout = ipw.Layout(width='98%',grid_gap='250px',align_items='center',border="solid 3px #7f003f", padding="35px") 

# Fourth element: Parameters Box
sectInput = ipw.VBox([paramsBox])
sectInput.layout = ipw.Layout(width='98%',border="solid 3px #7f003f", padding="35px")

# Fifth element: Run button
btnRun.layout = ipw.Layout(width='10%', height='80px', margin="90px")

# All elements together in a vertical box
webContent = ipw.VBox([introBox,hdrGuide,sectGuide,hdrDemographics,sectDemography,hdrParams,sectInput,btnRun, outputRun] ,layout={'align_items': 'center', 'width' : '90%', 'margin' : '80px 100px 80px 100px'})

# Final display
display(webTitle)
display(line)
display(webContent)